In [1]:

# 0. Install Dependencies and Bring in Data
# !pip install tensorflow tensorflow-gpu pandas matplotlib sklearn
# !pip list
# !pip install gradio jinja2

In [23]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow as tf
import gradio as gr
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [24]:
dfh = pd.read_csv('./hr.csv')
dft = pd.read_csv('./ttrain.csv')
df = pd.read_csv('./htrain.csv')
# df.head()
# ran=['toxic','severe_toxic','obscene','threat','insult','identity_hate']

In [25]:
dfh['funny']=dfh['humour'].apply(lambda x: 1 if x=='funny' else 0)
dfh['very_funny']=dfh['humour'].apply(lambda x: 1 if x=='very_funny' else 0)
dfh['hilarious']=dfh['humour'].apply(lambda x: 1 if x=='hilarious' else 0)
df.head(7000)

,text,humour
0,joe biden rule bid guy not run,0
1,watch darvish gave hitter whiplash slow pitch,0
2,call turtl without shell dead,1
3,reason elect feel person,0
4,pasco polic shot mexican migrant behind new au...,0
...,...,...
6995,call white guy surround court black peopl nba,1
6996,stephen colbert hobbit tv person make appear u...,0
6997,gymnast walk bar get twopoint deduct ruin chan...,1
6998,call uneven bodi water inconsistensea,1


In [30]:
Xh = df['ocr_text']
yh = dfh[dfh.columns[3:]].values
# Xt = dft['comment_text']
# yt = dft[df.columns[2:]].values
# li=np.append(Xh,Xt)
# lif=li.flatten()
# lst = li.tolist()

In [31]:
# 1. Preprocess
from tensorflow.keras.layers import TextVectorization
MAX_FEATURES = 200000 # number of words in the vocab
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')
vectorizer.adapt(Xh.values)
vectorized_text1 = vectorizer(Xh.values)
# vectorized_text2 = vectorizer(Xh.values)
# vectorized_text.shape

In [32]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text1, yh))
dataset = dataset.cache()
# dataset = dataset.shuffle(160000)
#dataset = dataset.shuffle(20002)
dataset = dataset.shuffle(7005)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks


In [33]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))


In [47]:
# # 2. Create Sequential Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))
# model.add(Dense(1, activation='sigmoid'))
model.compile(loss='BinaryCrossentropy', optimizer='Adam')
model.summary()


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 32)          6400032   
                                                                 
 bidirectional_4 (Bidirectio  (None, 64)               16640     
 nal)                                                            
                                                                 
 dense_16 (Dense)            (None, 128)               8320      
                                                                 
 dense_17 (Dense)            (None, 256)               33024     
                                                                 
 dense_18 (Dense)            (None, 128)               32896     
                                                                 
 dense_19 (Dense)            (None, 6)                 774       
                                                      

# Training model

In [45]:
history = model.fit(train, epochs=12, validation_data=val)

In [49]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model1.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

ValueError: Shapes (96,) and (0,) are incompatible

Socket exception: An existing connection was forcibly closed by the remote host (10054)


In [11]:
model1 = tf.keras.models.load_model('toxicitye5.h5')
model2 = tf.keras.models.load_model('rhumour.h5')
model3 = tf.keras.models.load_model('humour1.h5')

In [12]:
# input_text = vectorizer(['Thanks for being my best friend and always going along with my worst ideas'])
# # input_text.shape
# print(model.predict(input_text))
# # print(model2.predict(input_text))
# # (res >0.7).astype(int)

In [13]:
def score_comment(comment):
    input_text = vectorizer([comment])
    res1=(model1.predict(input_text))
    res2=(model2.predict(input_text))
    res3=(model3.predict(input_text))
    out=''
    l=[]
    a1=(res1 >=0.5).astype(int)
    for i in range(0,len(a1[0])):
        if a1[0][i]==1:
            l.append(i)
    if len(l)!=0:
        out += 'is_toxic : true\n'
        out += '{}: {}\n'.format('toxic_rate between 0 to 6', len(l))
    else:
        out += 'is_toxic : False\n'
    l=[]
    a2=(res2 >=0.5).astype(int)
    for i in range(1,len(a2[0])):
        if a2[0][i]==1:
            l.append(i)
    if res3[0][0]>0.7:
        try:
            c=max(l)
            out += 'is_humor : true\n'
            out += '{}: {}\n'.format('humor_rate between 0 to 3', c)
        except:
            out += 'is_humor : False\n'
    else:
        out += 'is_humor : False\n'
    return(out)

In [14]:
print(score_comment('I hate you'))

[[0.8306001  0.00651338 0.05816626 0.04909461 0.28580728 0.02899352]]
is_toxic : true
toxic_rate between 0 to 6: 1
is_humor : False



In [15]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

C:\Users\srini\anaconda3\lib\site-packages\gradio\inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\srini\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\srini\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


IMPORTANT: You are using gradio version 3.11.0, however version 3.14.0 is available, please upgrade.
--------


In [16]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://e097c0eab1ae45e5.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [17]:
# model.save('rhumour.h5')

In [18]:
# model8 = tf.keras.models.load_model('humour1.h5')

In [19]:
# input_text = vectorizer(['hi'])
# # input_text.shape
# print(model.predict(input_text))
# print(model8.predict(input_text))
# # (res >0.7).astype(int)

In [20]:
# def score_comment(comment):
# #     vectorized_comment = vectorizer([comment])
# # #     results = model.predict(vectorized_comment)
    
# #     text = ''
# #     for idx, col in enumerate(df.columns[2:]):
# #         text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
#     return comment

In [21]:
# interface = gr.Interface(fn=score_comment, 
#                          inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
#                         outputs='text')

In [22]:
# interface.launch(share=True)

[[0.8306001  0.00651338 0.05816626 0.04909461 0.28580728 0.02899352]]
